In [31]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders import WebBaseLoader
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, Runnable

import markdown
import datetime
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

import numpy as np

load_dotenv()

uri = os.getenv("MONGODB_URI")
client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
    

DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
URI = uri
collection = client[DB_NAME][COLLECTION_NAME]

Pinged your deployment. You successfully connected to MongoDB!


In [32]:
all_content_db = [x for x in collection.find()]

In [33]:
len(all_content_db)

19

In [34]:
original_content = [x['original_content'] for x in all_content_db]
summarization_content = [x['summarization'] for x in all_content_db]

In [35]:
model_name = "text-embedding-3-large"
embedding_size = 3072
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [36]:
embeddings_original_content = embeddings_model.embed_documents(
    original_content
)

embeddings_summarization_content = embeddings_model.embed_documents(
    summarization_content
)

In [37]:
embeddings_original_content_np = np.array(embeddings_original_content)
embeddings_summarization_content_np = np.array(embeddings_summarization_content)

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
matrix = cosine_similarity(embeddings_original_content_np, embeddings_summarization_content_np)
sims = np.array([matrix[i, i] for i in range(len(matrix))])

In [41]:
my_metric = (sims > 0.6).sum() / len(sims)

In [46]:
print("Resultado da métrica: {:.2f}%".format(my_metric * 100))

Resultado da métrica: 84.21%
